In [4]:
from imblearn.combine import *
from imblearn.over_sampling import *
from imblearn.under_sampling import *

In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import sklearn as sk
import datetime as dt

# import matplotlib as mpl
# mpl.use('Agg')
# import matplotlib.pylab as plt
# from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
from sklearn.preprocessing import Imputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
import seaborn as sns
sns.set()
sns.set_style("whitegrid")
sns.set_color_codes()
''
import warnings

warnings.filterwarnings('ignore')

In [73]:
aa = pd.read_csv('./coupon_data_project2/coupon_detail_train_translated_en.csv', parse_dates=['I_DATE'])


In [75]:
len(aa['USER_ID_hash'].unique())

22782

In [3]:
# train 
detail_train = pd.read_csv('./coupon_data_project2/coupon_detail_train_translated_en.csv', parse_dates=['I_DATE'])
visit_train = pd.read_csv('./coupon_data_project2/coupon_visit_train.csv', parse_dates=['I_DATE'])

area_train = pd.read_csv('./coupon_data_project2/coupon_area_train_translated_en.csv')
coupon_list_train = pd.read_csv('./coupon_data_project2/coupon_list_train_translated_en.csv', parse_dates=['DISPFROM', 'DISPEND', 'VALIDFROM', 'VALIDEND'])

# base data
location = pd.read_csv('./coupon_data_project2/train_location.csv')
user_list = pd.read_csv('./coupon_data_project2/user_list_translated_en.csv', parse_dates=['WITHDRAW_DATE', 'REG_DATE'])

# test data
area_test = pd.read_csv('./coupon_data_project2/test_location.csv')
coupon_list_test = pd.read_csv('./coupon_data_project2/coupon_list_test_translated_en.csv', parse_dates=['DISPFROM', 'DISPEND', 'VALIDFROM', 'VALIDEND'])

# submisiion
submission = pd.read_csv('./coupon_data_project2/sample_submission.csv')

In [68]:
submission

,USER_ID_hash,PURCHASED_COUPONS
0,0000b53e182165208887ba65c079fc21,NaN
1,00035b86e6884589ec8d28fbf2fe7757,NaN
2,0005b1068d5f2b8f2a7c978fcfe1ca06,NaN
3,000cc06982785a19e2a2fdb40b1c9d59,NaN
4,0013518e41c416cd6a181d277dd8ca0b,NaN
5,001acdee812a18acfd7509172bed5700,NaN
6,001fd7876e3aa29393537c6baf308e43,NaN
7,002383753c1e5d6305c8aff6f89e26d6,NaN
8,0025cae7997d25ea5cf8851bb099c798,NaN
9,002822059a01d895fad84f2f2ff5c1f1,NaN


In [5]:
# 테이블 현황 파악을 위한 함수 
def summary_table(table):
    df = pd.DataFrame()
    for i in table.columns:
        name = i
        dtype = table[i].dtype.name
        null = table[i].isnull().sum()
        act = table.shape[0] - null
        unique = len(table[i].unique())
        data = {'name': name, 'dtype': dtype, 'null': null, 'act': act, 'unique': unique}
        df = df.append(data, ignore_index=True)
    return df

## detail 만 detail_train으로 저장하기

## coupon list

In [8]:
coupon_list = pd.merge(coupon_list_test, coupon_list_train, how='outer')

In [9]:
coupon_list_test.shape, coupon_list_train.shape, coupon_list.shape

((310, 24), (19413, 24), (19723, 24))

In [10]:
# 캡슐과 장르 통합 및 명칭 변경
coupon_list['Case'] = coupon_list['CAPSULE_TEXT'] + coupon_list['GENRE_NAME']
coupon_list['Case'] = coupon_list['Case'].apply(lambda x: "HOTEL" if x == 'Guest houseHotel and Japanese hotel' 
                          or x == 'HotelHotel and Japanese hotel'
                          or x == 'Japanese hotelHotel and Japanese hotel'
                          or x == 'Japanse guest houseHotel and Japanese hotel'
                          or x == 'LodgeHotel and Japanese hotel'
                          or x == 'Public hotelHotel and Japanese hotel'
                          or x == 'Resort innHotel and Japanese hotel'
                          or x == 'Vacation rentalHotel and Japanese hotel'
                          else "NAIL" if x == 'Nail and eye salonNail and eye salon'
                          else "HAIR" if x == 'Hair salonHair salon'
                          else "FOOD" if x == 'FoodFood'
                          else "SPA" if x == 'SpaSpa'
                          else "BEAUTY" if x == 'BeautyBeauty'
                          else "CLASS" if x == 'ClassLesson'
                          else "CORRESPONDENCE" if x == 'Correspondence courseLessonClassLesson'
                          else "DELIVERY" if x == 'Delivery serviceDelivery service'
                          else "EVENT" if x == 'EventOther coupon'
                          else "GIFT" if x == 'Gift cardGift card'
                          else "HEALTH" if x == 'Health and medicalHealth and medical'
                          else "LEISURE" if x == 'LeisureLeisure'
                          else "LESSON" if x == 'LessonLesson'
                          else "OTHER" if x == 'OtherOther coupon'
                          else "RELAXATION" if x == 'RelaxationRelaxation'
                          else "WEB" if x == 'Web serviceOther coupon'
                          else 'OTHER'
                          )

In [11]:
# 실판매가 게산
coupon_list['Price'] = coupon_list['CATALOG_PRICE'] - coupon_list['DISCOUNT_PRICE']

In [12]:
# 실판매가 정규화
coupon_list["lnDPRICE"] = np.log1p(coupon_list["Price"])
coupon_list["mDPRICE"] = coupon_list.groupby("Case")["lnDPRICE"].transform(np.mean)
coupon_list["sDPRICE"] = coupon_list.groupby("Case")["lnDPRICE"].transform(np.std)
coupon_list["zprice"] = (coupon_list["lnDPRICE"] - coupon_list["mDPRICE"]) / coupon_list["sDPRICE"]

In [13]:
# 지역명 변경
coupon_list.rename(columns = {"LARGE_AREA_NAME": "spot_large", 
                              "ken_name": "spot_pref", 
                              "SMALL_AREA_NAME": "spot_small"}, inplace=True)

In [14]:
# usable: nan -> 1, 2 -> 0
coupon_list['USABLE_DATE_MON'].replace([0,2,1,np.nan],[0,0,1,1], inplace=True)
coupon_list['USABLE_DATE_TUE'].replace([0,2,1,np.nan],[0,0,1,1], inplace=True)
coupon_list['USABLE_DATE_WED'].replace([0,2,1,np.nan],[0,0,1,1], inplace=True)
coupon_list['USABLE_DATE_THU'].replace([0,2,1,np.nan],[0,0,1,1], inplace=True)
coupon_list['USABLE_DATE_FRI'].replace([0,2,1,np.nan],[0,0,1,1], inplace=True)
coupon_list['USABLE_DATE_SAT'].replace([0,2,1,np.nan],[0,0,1,1], inplace=True)
coupon_list['USABLE_DATE_SUN'].replace([0,2,1,np.nan],[0,0,1,1], inplace=True)
coupon_list['USABLE_DATE_HOLIDAY'].replace([0,2,1,np.nan],[0,0,1,1], inplace=True)
coupon_list['USABLE_DATE_BEFORE_HOLIDAY'].replace([0,2,1,np.nan],[0,0,1,1], inplace=True)

In [15]:
coupon_list.columns

Index(['CAPSULE_TEXT', 'GENRE_NAME', 'PRICE_RATE', 'CATALOG_PRICE',
       'DISCOUNT_PRICE', 'DISPFROM', 'DISPEND', 'DISPPERIOD', 'VALIDFROM',
       'VALIDEND', 'VALIDPERIOD', 'USABLE_DATE_MON', 'USABLE_DATE_TUE',
       'USABLE_DATE_WED', 'USABLE_DATE_THU', 'USABLE_DATE_FRI',
       'USABLE_DATE_SAT', 'USABLE_DATE_SUN', 'USABLE_DATE_HOLIDAY',
       'USABLE_DATE_BEFORE_HOLIDAY', 'spot_large', 'spot_pref', 'spot_small',
       'COUPON_ID_hash', 'Case', 'Price', 'lnDPRICE', 'mDPRICE', 'sDPRICE',
       'zprice'],
      dtype='object')

In [17]:
coupon_list['VALIDPERIOD'] = coupon_list['VALIDPERIOD'].fillna(179)

In [ ]:
# coupon_list.drop(labels = ['CAPSULE_TEXT'], axis=1, inplace=True )
# coupon_list.drop(labels = ['sDPRICE'], axis=1, inplace=True )

In [ ]:
# coupon_list.drop(labels = ['CAPSULE_TEXT'], axis=1, inplace=True )
# coupon_list.drop(labels = ['GENRE_NAME'], axis=1, inplace=True )
# coupon_list.drop(labels = ['CATALOG_PRICE'], axis=1, inplace=True )
# coupon_list.drop(labels = ['DISCOUNT_PRICE'], axis=1, inplace=True )
# coupon_list.drop(labels = ['DISPFROM'], axis=1, inplace=True )
# coupon_list.drop(labels = ['DISPEND'], axis=1, inplace=True )
# coupon_list.drop(labels = ['VALIDFROM'], axis=1, inplace=True )
# coupon_list.drop(labels = ['VALIDEND'], axis=1, inplace=True )
# coupon_list.drop(labels = ['lnDPRICE'], axis=1, inplace=True )
# coupon_list.drop(labels = ['mDPRICE'], axis=1, inplace=True )
# coupon_list.drop(labels = ['Price'], axis=1, inplace=True )
# coupon_list.drop(labels = ['spot_pref'], axis=1, inplace=True )  # 판단이슈 
# coupon_list.drop(labels = ['spot_small'], axis=1, inplace=True ) # 판단이슈 
# coupon_list.drop(labels = ['spot_large'], axis=1, inplace=True ) # 판단이슈 
# coupon_list.drop(labels = ['large_area_name'], axis=1, inplace=True ) # 판단이슈 

In [32]:
coupon_list.columns

Index(['CAPSULE_TEXT', 'GENRE_NAME', 'PRICE_RATE', 'CATALOG_PRICE',
       'DISCOUNT_PRICE', 'DISPFROM', 'DISPEND', 'DISPPERIOD', 'VALIDFROM',
       'VALIDEND', 'VALIDPERIOD', 'USABLE_DATE_MON', 'USABLE_DATE_TUE',
       'USABLE_DATE_WED', 'USABLE_DATE_THU', 'USABLE_DATE_FRI',
       'USABLE_DATE_SAT', 'USABLE_DATE_SUN', 'USABLE_DATE_HOLIDAY',
       'USABLE_DATE_BEFORE_HOLIDAY', 'spot_large', 'spot_pref', 'spot_small',
       'COUPON_ID_hash', 'Price', 'lnDPRICE', 'mDPRICE', 'sDPRICE', 'zprice',
       'Case_BEAUTY', 'Case_CLASS', 'Case_DELIVERY', 'Case_EVENT', 'Case_FOOD',
       'Case_GIFT', 'Case_HAIR', 'Case_HEALTH', 'Case_HOTEL', 'Case_LEISURE',
       'Case_LESSON', 'Case_NAIL', 'Case_OTHER', 'Case_RELAXATION', 'Case_SPA',
       'Case_WEB'],
      dtype='object')

In [20]:
# dummy 처리 할 column 명 넣어줌
ls_dummy = ['Case']

In [21]:
coupon_list = pd.get_dummies(coupon_list, columns = ls_dummy)

In [22]:
# train set과 test set을 다시 분리
coupon_list_train = coupon_list[311:]
coupon_list_test = coupon_list[:310]

# detail

In [44]:
detail_train = pd.read_csv('./coupon_data_project2/coupon_detail_train_translated_en.csv', parse_dates=['I_DATE'])


In [45]:
detail_train.rename(columns = {'I_DATE': 'purchase_date'}, inplace=True)
detail_train.rename(columns = {'SMALL_AREA_NAME': 'resid_small'}, inplace=True)

In [46]:
detail_train.drop(labels = ['ITEM_COUNT'], axis=1, inplace=True)
detail_train.drop(labels = ['PURCHASEID_hash'], axis=1, inplace=True)
detail_train.drop(labels = ['resid_small'], axis=1, inplace=True)
detail_train.drop(labels = ['purchase_date'], axis=1, inplace=True)

In [47]:
detail_train.head()

,USER_ID_hash,COUPON_ID_hash
0,d9dca3cb44bab12ba313eaa681f663eb,34c48f84026e08355dc3bd19b427f09a
1,560574a339f1b25e57b0221e486907ed,767673b7a777854a92b73b0934ddfae7
2,560574a339f1b25e57b0221e486907ed,4f3b5b91d9831192557c056022fdc1f2
3,560574a339f1b25e57b0221e486907ed,4f3b5b91d9831192557c056022fdc1f2
4,560574a339f1b25e57b0221e486907ed,4f3b5b91d9831192557c056022fdc1f2


In [48]:
detail_train = pd.merge(detail_train, coupon_list, how='left', on='COUPON_ID_hash')

In [49]:
detail_train.columns

Index(['USER_ID_hash', 'COUPON_ID_hash', 'CAPSULE_TEXT', 'GENRE_NAME',
       'PRICE_RATE', 'CATALOG_PRICE', 'DISCOUNT_PRICE', 'DISPFROM', 'DISPEND',
       'DISPPERIOD', 'VALIDFROM', 'VALIDEND', 'VALIDPERIOD', 'USABLE_DATE_MON',
       'USABLE_DATE_TUE', 'USABLE_DATE_WED', 'USABLE_DATE_THU',
       'USABLE_DATE_FRI', 'USABLE_DATE_SAT', 'USABLE_DATE_SUN',
       'USABLE_DATE_HOLIDAY', 'USABLE_DATE_BEFORE_HOLIDAY', 'spot_large',
       'spot_pref', 'spot_small', 'Price', 'lnDPRICE', 'mDPRICE', 'sDPRICE',
       'zprice', 'Case_BEAUTY', 'Case_CLASS', 'Case_DELIVERY', 'Case_EVENT',
       'Case_FOOD', 'Case_GIFT', 'Case_HAIR', 'Case_HEALTH', 'Case_HOTEL',
       'Case_LEISURE', 'Case_LESSON', 'Case_NAIL', 'Case_OTHER',
       'Case_RELAXATION', 'Case_SPA', 'Case_WEB'],
      dtype='object')

In [33]:
# 2. replace hash string ---------------------------------
user_ids = user_list.USER_ID_hash
coupon_train_ids = coupon_list_train.COUPON_ID_hash

# for test coupon, added 1000000 to IDs
coupon_test_ids = coupon_list_test.COUPON_ID_hash
coupon_test_ids.rename(lambda x : x + 1000000, inplace=True)
coupon_ids = pd.concat([coupon_train_ids, coupon_test_ids])

def replace_hash(hashes, hash_id_table):
    replace_table = pd.Series(hash_id_table.index, index=hash_id_table.values)
    return replace_table[hashes].values

# replace hash and drop extra columns
detail = detail_train.copy()
detail["USER_ID"] = replace_hash(detail.USER_ID_hash, user_ids)
detail["COUPON_ID"] = replace_hash(detail.COUPON_ID_hash, coupon_ids)
detail = detail[["USER_ID", "COUPON_ID"]]

In [34]:
len(np.unique(coupon_list_train.COUPON_ID_hash.values))

19412

In [50]:
detail_train.columns

Index(['USER_ID_hash', 'COUPON_ID_hash', 'CAPSULE_TEXT', 'GENRE_NAME',
       'PRICE_RATE', 'CATALOG_PRICE', 'DISCOUNT_PRICE', 'DISPFROM', 'DISPEND',
       'DISPPERIOD', 'VALIDFROM', 'VALIDEND', 'VALIDPERIOD', 'USABLE_DATE_MON',
       'USABLE_DATE_TUE', 'USABLE_DATE_WED', 'USABLE_DATE_THU',
       'USABLE_DATE_FRI', 'USABLE_DATE_SAT', 'USABLE_DATE_SUN',
       'USABLE_DATE_HOLIDAY', 'USABLE_DATE_BEFORE_HOLIDAY', 'spot_large',
       'spot_pref', 'spot_small', 'Price', 'lnDPRICE', 'mDPRICE', 'sDPRICE',
       'zprice', 'Case_BEAUTY', 'Case_CLASS', 'Case_DELIVERY', 'Case_EVENT',
       'Case_FOOD', 'Case_GIFT', 'Case_HAIR', 'Case_HEALTH', 'Case_HOTEL',
       'Case_LEISURE', 'Case_LESSON', 'Case_NAIL', 'Case_OTHER',
       'Case_RELAXATION', 'Case_SPA', 'Case_WEB'],
      dtype='object')

## 필요없는 변수 지우기

In [51]:
del_ls = ['spot_large', 'CAPSULE_TEXT', 'GENRE_NAME','VALIDFROM', 'VALIDEND', 'VALIDPERIOD',
       'spot_pref', 'spot_small', 'Price', 'lnDPRICE', 'mDPRICE', 'sDPRICE']
train = detail_train.drop(del_ls, 1)

In [62]:
train.columns

Index(['USER_ID_hash', 'COUPON_ID_hash', 'PRICE_RATE', 'CATALOG_PRICE',
       'DISCOUNT_PRICE', 'DISPFROM', 'DISPEND', 'DISPPERIOD',
       'USABLE_DATE_MON', 'USABLE_DATE_TUE', 'USABLE_DATE_WED',
       'USABLE_DATE_THU', 'USABLE_DATE_FRI', 'USABLE_DATE_SAT',
       'USABLE_DATE_SUN', 'USABLE_DATE_HOLIDAY', 'USABLE_DATE_BEFORE_HOLIDAY',
       'zprice', 'Case_BEAUTY', 'Case_CLASS', 'Case_DELIVERY', 'Case_EVENT',
       'Case_FOOD', 'Case_GIFT', 'Case_HAIR', 'Case_HEALTH', 'Case_HOTEL',
       'Case_LEISURE', 'Case_LESSON', 'Case_NAIL', 'Case_OTHER',
       'Case_RELAXATION', 'Case_SPA', 'Case_WEB'],
      dtype='object')

### test 생성

In [63]:
test = coupon_list_test.filter(train.columns)

In [65]:
compare_not_test = [i for i in train.columns if i not in test.columns]
compare_not_train = [i for i in test.columns if i not in train.columns]
print('only_train: {}  \n'.format(compare_not_test))
print('only_test: {}'.format(compare_not_train))

only_train: ['USER_ID_hash']  

only_test: []


## 유사도 구하기

In [66]:
train_drop_c = train.drop('COUPON_ID_hash', axis=1)
user = train_drop_c.groupby(by='USER_ID_hash').apply(np.mean)

In [57]:
test_features = test.drop('COUPON_ID_hash', axis=1)
test_features.columns

Index(['CAPSULE_TEXT', 'GENRE_NAME', 'PRICE_RATE', 'CATALOG_PRICE',
       'DISCOUNT_PRICE', 'DISPFROM', 'DISPEND', 'DISPPERIOD', 'VALIDFROM',
       'VALIDEND', 'VALIDPERIOD', 'USABLE_DATE_MON', 'USABLE_DATE_TUE',
       'USABLE_DATE_WED', 'USABLE_DATE_THU', 'USABLE_DATE_FRI',
       'USABLE_DATE_SAT', 'USABLE_DATE_SUN', 'USABLE_DATE_HOLIDAY',
       'USABLE_DATE_BEFORE_HOLIDAY', 'spot_large', 'spot_pref', 'spot_small',
       'Price', 'lnDPRICE', 'mDPRICE', 'sDPRICE', 'zprice', 'Case_BEAUTY',
       'Case_CLASS', 'Case_DELIVERY', 'Case_EVENT', 'Case_FOOD', 'Case_GIFT',
       'Case_HAIR', 'Case_HEALTH', 'Case_HOTEL', 'Case_LEISURE', 'Case_LESSON',
       'Case_NAIL', 'Case_OTHER', 'Case_RELAXATION', 'Case_SPA', 'Case_WEB'],
      dtype='object')

In [ ]:
from sklearn.preprocessing import *

In [58]:
user.shape, test_features.shape

((22782, 30), (310, 44))

In [59]:
user.isnull().sum() , \
test_features.isnull().sum()

(PRICE_RATE                    0
 CATALOG_PRICE                 0
 DISCOUNT_PRICE                0
 DISPPERIOD                    0
 USABLE_DATE_MON               0
 USABLE_DATE_TUE               0
 USABLE_DATE_WED               0
 USABLE_DATE_THU               0
 USABLE_DATE_FRI               0
 USABLE_DATE_SAT               0
 USABLE_DATE_SUN               0
 USABLE_DATE_HOLIDAY           0
 USABLE_DATE_BEFORE_HOLIDAY    0
 zprice                        0
 Case_BEAUTY                   0
 Case_CLASS                    0
 Case_DELIVERY                 0
 Case_EVENT                    0
 Case_FOOD                     0
 Case_GIFT                     0
 Case_HAIR                     0
 Case_HEALTH                   0
 Case_HOTEL                    0
 Case_LEISURE                  0
 Case_LESSON                   0
 Case_NAIL                     0
 Case_OTHER                    0
 Case_RELAXATION               0
 Case_SPA                      0
 Case_WEB                      0
 dtype: in

In [72]:
len(train['USER_ID_hash'].unique())

22782

In [ ]:
vector_1 = user.copy()
vector_2 = test_features.copy()

vector_1 = scale(vector_1)
vector_1 = pd.DataFrame(data = vector_1, index = user.index, columns = user.columns )

vector_2 = scale(vector_2)
vector_2 = pd.DataFrame(data = vector_2, index = test_features.index, columns = test_features.columns )

In [ ]:
import math

def Cosine(vec1, vec2) :
    result = InnerProduct(vec1,vec2) / (VectorSize(vec1) * VectorSize(vec2))
    return result

def VectorSize(vec) :
    return math.sqrt(sum(math.pow(v,2) for v in vec))

def InnerProduct(vec1, vec2) :
    return sum(v1*v2 for v1,v2 in zip(vec1,vec2))

def Euclidean(vec1, vec2) :
    return math.sqrt(sum(math.pow((v1-v2),2) for v1,v2 in zip(vec1, vec2)))

def Theta(vec1, vec2) :
    return math.acos(Cosine(vec1,vec2)) + 10

def Triangle(vec1, vec2) :
    theta = math.radians(Theta(vec1,vec2))
    return (VectorSize(vec1) * VectorSize(vec2) * math.sin(theta)) / 2

def Magnitude_Difference(vec1, vec2) :
    return abs(VectorSize(vec1) - VectorSize(vec2))

def Sector(vec1, vec2) :
    ED = Euclidean(vec1, vec2)
    MD = Magnitude_Difference(vec1, vec2)
    theta = Theta(vec1, vec2)
    return math.pi * math.pow((ED+MD),2) * theta/360

def TS_SS(vec1, vec2) :
    return Triangle(vec1, vec2) * Sector(vec1, vec2)

vec1 = [1,2]
vec2 = [2,4]

print(Euclidean(vec1,vec2))
print(Cosine(vec1,vec2))
print(TS_SS(vec1,vec2))

In [ ]:
from scipy import spatial
def cosine_similarity(vector_1, vector_2):

    return 1 - spatial.distance.cosine(vector_1, vector_2)

In [ ]:
matrix = []
for idx1 in tqdm(range(len(user))):
    
    row = []
    for idx2 in range(len(test_features)):
        row.append(cosine_similarity(vector_1.iloc[idx1], vector_2.iloc[idx2]))
    matrix.append(row)

In [ ]:
continuous = list(set(vector_1) - set(vector_2))
continuous

In [ ]:
coupons_ids = test['COUPON_ID_hash']
result_index = vector_1.index
result_columns = [coupons_ids[i] for i in range(0, 310)]
result_df = pd.DataFrame(index= result_index, columns=result_columns,
                      data=matrix)

In [ ]:
submission = pd.DataFrame(columns=['USER_ID_hash','PURCHASED_COUPONS'])
submission['USER_ID_hash'] = result_df.index
submission['PURCHASED_COUPONS'] = result_df.T.apply(lambda x: " ".join(x.sort_values(ascending=False)[:21].index)).values

In [ ]:
submission.to_csv('submission/cosine.csv',index = False,header=True)
